# Strands Agentsレスポンスの高度な処理

Strands Agentsでは、エージェント実行中に発生するイベントを傍受して処理するための2つの方法が提供されています：
    
- **非同期イテレータ**: FastAPI、aiohttp、Django Channelsなどの非同期フレームワークに最適です。これらの環境では、SDKは非同期イテレータを返す`stream_async`メソッドを提供します。
- **コールバックハンドラ**: エージェント実行中に発生するイベントを傍受して処理できます。これにより、リアルタイム監視、カスタム出力フォーマット、外部システムとの統合が可能になります。

この例では、両方の方法を使用してエージェントへの呼び出しを処理する方法を示します


## エージェントの詳細
<div style="float: left; margin-right: 20px; ">
    
|機能                |説明                                        |
|--------------------|-------------------------------------------|
|使用する機能         |非同期イテレータ、コールバックハンドラ         |
|エージェント構造     |シングルエージェントアーキテクチャ              |
|使用するネイティブツール   |calculator                           |
|作成するカスタムツール|天気予報                                   |

</div>

## アーキテクチャ

<div style="text-align:left;">
    <img src="images/architecture.png" width="65%" />
</div>

## 主な機能
* ストリーミング用の非同期イテレータ
* コールバックハンドラ


## セットアップと前提条件

### 前提条件
* Python 3.10以上
* AWSアカウント
* Amazon BedrockでAnthropic Claude 3.7が有効化されていること

それでは、Strands Agent Agentに必要なパッケージをインストールしましょう

In [ ]:
# installing pre-requisites
!pip install -r requirements.txt

### 依存パッケージのインポート

それでは、依存パッケージをインポートしましょう

In [ ]:
import asyncio

import httpx
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
from strands import Agent, tool
from strands_tools import calculator

## 方法1 - ストリーミング用の非同期イテレータ


Strands Agentsは、`stream_async`メソッドを通じて非同期イテレータのサポートを提供し、Webサーバー、API、その他の非同期アプリケーションなどの非同期環境でエージェントレスポンスのリアルタイムストリーミングを可能にします。

この例をノートブックで紹介しているため、`asyncio.run`と`loop.run_until_complete`のネストされた使用を許可するために`nest_asyncio`を適用する必要があります

In [ ]:
nest_asyncio.apply()

### stream_asyncを使用したエージェントの作成と呼び出し

それでは、組み込みcalculatorツールと`callback_handler`なしでエージェントを作成しましょう。`stream_async`メソッドを使用して、ストリーミングされたエージェントイベントを反復処理します

In [ ]:
# Initialize our agent without a callback handler
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    tools=[calculator], 
    callback_handler=None)

# Async function that iterators over streamed agent events


async def process_streaming_response():
    agent_stream = agent.stream_async("Calculate 2+2")
    async for event in agent_stream:
        print(event)


# Run the agent
asyncio.run(process_streaming_response())

###  イベントループのライフサイクルの追跡

この例は、イベントループのライフサイクルと、イベントが互いにどのように関連しているかを示しています。Strands Agentでの実行フローを理解するのに役立ちます：

エージェントストリームイベントをより適切に分析するために、印刷フォーマットコードを作成しましょう。同じエージェントを引き続き使用します

In [ ]:
# Async function that iterators over streamed agent events


async def process_streaming_response():
    agent_stream = agent.stream_async("What is the capital of France and what is 42+7?")
    async for event in agent_stream:
        # Track event loop lifecycle
        if event.get("init_event_loop", False):
            print("🔄 Event loop initialized")
        elif event.get("start_event_loop", False):
            print("▶️ Event loop cycle starting")
        elif event.get("start", False):
            print("📝 New cycle started")
        elif "message" in event:
            print(f"📬 New message created: {event['message']['role']}")
        elif event.get("force_stop", False):
            print(
                f"🛑 Event loop force-stopped: {event.get('force_stop_reason', 'unknown reason')}"
            )

        # Track tool usage
        if "current_tool_use" in event and event["current_tool_use"].get("name"):
            tool_name = event["current_tool_use"]["name"]
            print(f"🔧 Using tool: {tool_name}")

        # Show only a snippet of text to keep output clean
        if "data" in event:
            # Only show first 20 chars of each chunk for demo purposes
            data_snippet = event["data"][:20] + (
                "..." if len(event["data"]) > 20 else ""
            )
            print(f"📟 Text: {data_snippet}")

    return event["result"]


# Run the agent
asyncio.run(process_streaming_response())

### FastAPI統合

`stream_async`をFastAPIと統合して、アプリケーションへのストリーミングエンドポイントを作成することもできます。このために、エージェントに`weather_forecast`ツールを追加します。アーキテクチャの更新は以下のようになります

<div style="text-align:left;">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [ ]:
# Tool definition


@tool
def weather_forecast(city: str, days: int = 3) -> str:
    return f"Weather forecast for {city} for the next {days} days..."


# FastAPI app
app = FastAPI()


class PromptRequest(BaseModel):
    prompt: str


@app.post("/stream")
async def stream_response(request: PromptRequest):
    async def generate():
        agent = Agent(tools=[calculator, weather_forecast], callback_handler=None)
        try:
            async for event in agent.stream_async(request.prompt):
                if "data" in event:
                    yield event["data"]
        except Exception as e:
            yield f"Error: {str(e)}"

    return StreamingResponse(generate(), media_type="text/plain")


# Function to start server without blocking


async def start_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8001, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()


# Run server as background task
if "server_task" not in globals():
    server_task = asyncio.create_task(start_server())
    await asyncio.sleep(0.1)  # Give server time to start

print("✅ Server is running at http://0.0.0.0:8001")

#### FastAPIエージェントの呼び出し
そして、プロンプトでエージェントを呼び出すことができます

In [ ]:
async def fetch_stream():
    async with httpx.AsyncClient() as client:
        async with client.stream(
            "POST",
            "http://0.0.0.0:8001/stream",
            json={"prompt": "What is weather in NYC?"},
        ) as response:
            async for line in response.aiter_lines():
                if line.strip():  # Skip empty lines
                    print("Received:", line)


await fetch_stream()

## 方法2 - ストリーミング用のコールバックハンドラ

コールバックハンドラは、Strands Agentsの強力な機能で、エージェント実行中に発生するイベントを傍受して処理できます。これにより、リアルタイム監視、カスタム出力フォーマット、外部システムとの統合が可能になります。



コールバックハンドラは、エージェントのライフサイクル中に発生するイベントをリアルタイムで受信します：

- モデルからのテキスト生成
- ツールの選択と実行
- 推論プロセス
- エラーと完了


それでは、ツール使用とモデル出力を強調するためにイベント入力をフォーマットするカスタムコールバックハンドラ関数を作成しましょう。そのために、calculatorツールのみを持つエージェントを再び使用します

<div style="text-align:left;">
    <img src="images/architecture.png" width="65%" />
</div>

In [ ]:
def custom_callback_handler(**kwargs):
    # Process stream data
    if "data" in kwargs:
        print(f"MODEL OUTPUT: {kwargs['data']}")
    elif "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
        print(f"\nUSING TOOL: {kwargs['current_tool_use']['name']}")


# Create an agent with custom callback handler
agent = Agent(   
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
        tools=[calculator], 
        callback_handler=custom_callback_handler)

agent("Calculate 2+2")

### おめでとうございます！

このノートブックでは、非同期イテレータとコールバックハンドラを使用してエージェントの出力をストリーミングする方法を学びました。